In [12]:
import os
import re
import pandas as pd
import json
import random
import jieba


In [13]:
def __merge_symmetry(sentences, symmetry=('“','”')):
    '''合并对称符号，如双引号'''
    effective_ = []
    merged = True
    for index in range(len(sentences)):       
        if symmetry[0] in sentences[index] and symmetry[1] not in sentences[index]:
            merged = False
            effective_.append(sentences[index])
        elif symmetry[1] in sentences[index] and not merged:
            merged = True
            effective_[-1] += sentences[index]
        elif symmetry[0] not in sentences[index] and symmetry[1] not in sentences[index] and not merged :
            effective_[-1] += sentences[index]
        else:
            effective_.append(sentences[index])
        
    return [i.strip() for i in effective_ if len(i.strip()) > 0]

def to_sentences(paragraph):
    """由段落切分成句子"""
    sentences = re.split(r"(？|。|！|\…\…)", paragraph)
    sentences.append("")
    sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
    sentences = [i.strip() for i in sentences if len(i.strip()) > 0]
    
    for j in range(1, len(sentences)):
        if sentences[j][0] == '”':
            sentences[j-1] = sentences[j-1] + '”'
            sentences[j] = sentences[j][1:]
        elif sentences[j][0] == '"' and (sentences[j-1].count('"') % 2 != 0):
            sentences[j-1] = sentences[j-1] + '"'
            sentences[j] = sentences[j][1:]
            
    sentences = __merge_symmetry(sentences, symmetry=('“','”'))
    sentences = __merge_symmetry(sentences, symmetry=('(',')'))
    sentences = __merge_symmetry(sentences, symmetry=('（','）'))
    return sentences

In [14]:
input_file = 'C:/Users/ly-lhr.PVI/车型识别模型/模型构建/东风日产负面舆情日报2020年6月30日9：30/东风日产负面舆情日报2020年6月30日9：30.xlsx'
output_dir = input_file[:-5]

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [15]:
df = pd.read_excel(input_file, sheet_name='网络')[['ID', '主要内容']]

In [16]:
df_seg = []
for i, row in df.iterrows():
    id, content = row['ID'], row['主要内容']
    paragraph_id = 0
    for paragraph in content.split('\n'):
        paragraph = paragraph.strip()
        if paragraph:
            paragraph_id += 1
            sent_id = 0
            for sent in to_sentences(paragraph):
                sent = sent.strip()
                if sent:
                    sent_id += 1
                    df_seg.append([id, paragraph_id, sent_id, sent])
df_seg = pd.DataFrame(df_seg, columns=['ID', 'paragraph_id', 'sent_id', 'sentence'])

In [23]:
df_seg.to_csv('C:/Users/ly-lhr.PVI/车型识别模型/模型构建/东风日产负面舆情日报2020年6月30日9：30/tmp.csv', encoding='utf-8-sig')

In [24]:
df.head()

,ID,主要内容
0,1,"阅读本文前，请您先点击上面的蓝色字体""汽车大狮""，再点击""关注""，这样您就可以继续免费收到内..."
1,2,汽车是一个机器，它的车身是由钢铁制成的，如果防锈防腐做不好的话，车身就很容易会生锈。有一些汽...
2,3,据相关汽车销售网统计，东风日产天籁单月销量为6755辆，1-累计销量为20599辆，无论是月...
3,4,可变压缩比发动机那么厉害，为何天籁还是凉？\r\n日系三杰，丰田凯美瑞、本田雅阁和日产天籁。...
4,5,日产途达的竞争目标是丰田普拉多，刚上市的时候，信誓旦旦会把普拉多给打倒，但是最终证明普拉多是...


In [25]:
s = """
很多人买车还是非常挑剔的，都希望经过一番精挑细选后能买到适合自己的车型，这也容易理解。首先其次的价格并不便宜，一辆十来万也是正常的，贵的二三十万甚至上百万，而普通人买车自然更关注汽车的性价比，谁也不想花钱买了不喜欢的车型。此外，对于一些追求个性的人来说，普通的家轿似乎并不能满足日常需求，因此就会考虑买越野车或者是轿跑车之类的，尤其是越野车，在当下也是越来越受欢迎。
打开APP，查看更多精彩图片
而在越野车这个市场中，耳熟能详的车型也不少，像吉普牧马人、奔驰G级、丰田陆巡、日产途乐等，当然，低端市场也有一些比较有名气的车型，像北汽212、铃木吉姆尼等。其实，越野的门槛还是比较高的，普通人如果真的要玩越野，那基本不可能买得起几十万的越野车。因此，日产也是看到了这个市场的前景，推出了一款比较便宜的硬派SUV，又一硬派SUV倒下，尺寸媲美汉兰达，起步16万多，可惜没人懂，该车便是日产途达。
不可否认，途达作为硬派SUV还是有资格的，首先就是外观，整体风格和老大哥途乐有些类似，看上去比较方正硬朗，这也是符合硬派SUV的定位。该车前脸采用了日产最终的设计，大尺寸V型前脸加入大量镀铬饰条修饰，让整车看上去更有档次感。而在车身侧面，途达同样也是和其身份相符，方正的车身，没有太多腰线的修饰，但这并不重要，毕竟硬派SUV要的就是这种""狂野""，并不需要过多的腰线，那样反而会让其看起来小家子气。
车尾的造型比较普通，和国产硬派SUV哈弗H9比起来还是差距还是比较明显的，不过简洁的设计也意料之内。值得一提的是，其离去角达到了27度，这样一来车尾比较高，下方大量黑色饰板的修饰，在视觉上无疑更有冲击力。而在尺寸方面，途达也是有较大的优势，其车长达到了4882mm，车高为1835mm，并且还有着2850mm的轴距。这样的车身尺寸，完全可以媲美汉兰达，在气势上确实能给人很强的满足感。
来到车内，途达内饰的设计相对于外观就比较一般了，并没有什么让人眼前一亮的地方，其中方向盘采用日产经典的三幅式设计，不过造型并不太符合国人的审美。此外，对称式中控台的造型没有太多花里胡哨的设计，考虑到其定位，这也算在情理之中。但做工用料就比较一般了，中控台基本使用的是硬塑料，给人的感觉比较廉价。但空间布局还是比较合理的，储物格数量较多，而且尺寸都比较大，完全不用担心存在问题。当然，途达的座椅也是比较柔软，这也符合日产一贯的风格，但后排靠背角度比较直，长时间乘坐并不怎么舒服。
而在动力方面，途达全系搭载的是一台代号为QR25的2.5L发动机，最大马力193匹，峰值扭矩245牛米，匹配6挡手动和7挡手自一体变速箱。不难发现，这台2.5L发动机和奇骏上的那台2.5L是同一系列的发动机，只是调教有些许差异。当然，对于有着近两吨车重的途达而言，这套动力组合其实并不怎么够用，当然越野的话还是可以胜任的。在高配车型上，途达配备了分时四驱系统，其结构和普拉多一样，在低速下可以将扭矩放大2.7倍，这样就能大幅度提高脱困能力了。
其实，对于途达而言，日产的算盘还是非常明确的，那就是走低端路线，该车指导价为16.98-24.83万，而且市面上还有几千上万的优惠，入门车型起步价在16万左右，性价比还是非常高的。只是遗憾的是途达的销量异常惨淡，该车在五月份卖出了1185台，六月份更是只卖出了1021台，每月一千多的销量确实让人没想到。当然，这和该车采用后轮鼓刹有关，毕竟鼓刹在性能和稳定性上确实不如通风盘式刹车系统，而行车安全，无疑是最重要的。"
""" 

In [26]:
s = """
很多人买车还是非常挑剔的，都希望经过一番精挑细选后能买到适合自己的车型，这也容易理解。"""

In [27]:
s = s.strip()
T = [[[sent, ''] for sent in to_sentences(p)] for p in s.split('\n')]
T

[[['很多人买车还是非常挑剔的，都希望经过一番精挑细选后能买到适合自己的车型，这也容易理解。', '']]]

In [22]:
class Sentence:
    def __init__(self, text, car=[]):
        self.text = text
        self.car = car

class Paragraph:
    def __init__(self, car=[]):
        self.car = car
        self.sentence = []
    def add_sentence(self, sentence):
        self.sentence.append(sentence)
    def update_car(self):
        car = set()
        for sent in self.sentence:
            car.update(sent.car)
        self.car = car
    def infer_sent(self):
        pass
    def print_paragraph(self):
        paragraph = ''.join([sent.text for sent in self.sentence])
        print(paragraph)

class Article:
    def __init__(self, title, car=[]):
        self.title = title
        self.car = car
        self.paragraph = []
    def add_paragraph(self, paragraph):
        self.paragraph.append(paragraph)
    def update_car(self):
        car = set()
        for para in self.paragraph:
            para.update_car()
            car.update(para.car)
        self.car = car
    def infer_paragraph(self):
        pass
    def print_article(self):
        print('Title:', self.title)
        print('Content:')
        for paragraph in self.paragraph:
            paragraph.print_paragraph()
    def save_article(self):
        pass
    
